In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import PIL

import configparser as ConfigParser
import numpy as np
import matplotlib.pyplot as plt

import utils
from networks import AGRadGalNet
from datasets import MiraBest_full, MBFRConfident, MBFRUncertain, MBHybrid

# Set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)

# Get correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read in config file
config_name = "configs/template.cfg"
config = ConfigParser.ConfigParser(allow_no_value=True)
config.read(config_name)

# Load network architecture (with random weights)
print(f"Loading in {config['model']['base']}")
net = locals()[config['model']['base']](**config['model'])

Loading in AGRadGalNet


In [28]:
# Create data transformations
datamean = config.getfloat('data', 'datamean')
datastd = config.getfloat('data', 'datastd')
number_rotations = config.getint('data', 'number_rotations')
imsize = config.getint('data', 'imsize')
scaling_factor = config.getfloat('data', 'scaling')
angles = list(range(0, 360, config.getint('data', 'number_rotations')))
p_flip = 0.5 if config.getboolean('data','flip') else 0

# Create hard random (seeded) rotation:
class RotationTransform:
    """Rotate by one of the given angles."""
    def __init__(self, angles, resample):
        self.angles = angles
        self.resample = resample

    def __call__(self, x):
        angle = np.random.choice(a=self.angles, size=1)
        return transforms.functional.rotate(x, angle, resample=self.resample)


# Compose dict of transformations
transformations = {
    'none': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ]),
    'rotation and flipping': transforms.Compose([
        transforms.CenterCrop(imsize),
        transforms.RandomVerticalFlip(p=p_flip),
        RotationTransform(angles, resample=PIL.Image.BILINEAR),
        transforms.RandomAffine(
            degrees=0, # No uncontrolled rotation
            scale=(1-scaling_factor, 1+scaling_factor), 
            resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ]),
    'no rotation no flipping': transforms.Compose([
        transforms.CenterCrop(imsize),
        transforms.RandomVerticalFlip(p=p_flip),
        transforms.RandomAffine(
            degrees=0, # No uncontrolled rotation
            scale=(1-scaling_factor, 1+scaling_factor), 
            resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ])
}

# Read / Create Folder for Data to be Saved
root = config['data']['directory']
if not os.path.isdir(root):
    os.mkdir(root)
download = True
train = True
data_class = locals()[config['data']['dataset']]
data_set = data_class(root=root, download=download, train=train, transform=transformations['rotation and flipping'])

# Seperate Data into Subsets for Training, Validation, Testing.
traindata = data_class(root=root, download=download, train=True, transform=transformations['rotation and flipping'])
testdata = data_class(root=root, download=download, train=False, transform=transformations['rotation and flipping'])

trainset = torch.utils.data.DataLoader(traindata, batch_size=config.getint('training', 'batch_size'))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [36]:
count = 0
for data, label in traindata:
    count+=1
print(count)

for d in trainset:
    print(d)
    break
    
print(traindata.data.shape)
print(len(traindata.targets))
print(traindata[10:100][0].shape)
plt.imshow(traindata[0][0].squeeze())
plt.show()

1099
[tensor([[[[-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          ...,
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886]]],


        [[[-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          ...,
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886]]],


        [[[-0.0886, -0.0886, -0.0886,  ..., -0.0886, -0.0886, -0.0886],
          [-0.0886, 

ValueError: cannot reshape array of size 2025000 into shape (150,150)

In [14]:
learning_rate = config.getfloat('training', 'learning_rate')
optimizers = {
    'SGD': optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9),
    'Adagrad': optim.Adagrad(net.parameters(), lr=learning_rate),
    'Adadelta': optim.Adadelta(net.parameters(), lr=learning_rate),
    'Adam': optim.Adam(net.parameters(), lr=learning_rate)
}


1e-05


Data Manipulation:
- path_to_model(file_name)
- data_call(dataset_name)
- determine_dataset(dataset,model_name) ... dataset in ['automatic','FRDEEP-F','MiraBest']

Model Manipulation:
- load_net(model_name,device)
- training_validation(PATH,xlims=[None,None],save=False,full_path=False) ... PATH is a local title of a folder or file (within ./TrainedNetworks)
- prediction(dataset, net, class_groups,(device='cuda',reps='360'))
- evaluate(file_name, dataset='automatic')

Evaluation Plots:
- plot_conf_mat(conf_matrix,normalised=True,n_classes=2,format_input=None,title='Confusion Matrix')
- plot_roc_curve(fpr,tpr,title='ROC Curve (AUC=\{auc:.3f\})')
- out_print(out)

Attention Maps:
- attentions_func(batch_of_images, net, mean=True, device=torch.device('cpu'))
- attention_analysis(source, source_only=True, attention_maps=None, GradCAM=None)
- AttentionImagesByEpoch(sources, folder_name, net,epoch=1500, device=torch.device('cpu'))
- attention_epoch_plot(source_images,folder_name, logged=False, width=3, device=torch.device('cpu'))

GradCAM:
- To be completed.

Other:
- mask_on_image(img, mask)
- SortedDataSamples(data_name, transformed=True,  rotations=1, subset='NOHYBRID')
- net_name_extraction(PATH)

Incomplete:
- Loading from Pickled dicts
- GradCAM Call for a given image